In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch
import torch.nn.functional as F
import time
from util import findStatistics
from sklearn.preprocessing import LabelEncoder

In [34]:
le = LabelEncoder()
df=pd.read_csv(r"C:\Users\User\Desktop\מדמח\מודלימז\פרויקט חדש\archive (3)\full_data.csv")

X=df.apply(le.fit_transform)
X=X.drop(columns=['stroke'],axis=0)
y=df['stroke']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [62]:
X


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,1,88,0,1,1,1,1,3769,225,1
1,1,101,0,1,1,1,0,2444,184,2
2,0,70,0,0,1,1,1,3324,203,3
3,0,100,1,0,1,2,0,3333,99,2
4,1,102,0,0,1,1,1,3386,149,1
...,...,...,...,...,...,...,...,...,...,...
4976,1,62,0,0,0,1,0,576,157,1
4977,1,61,0,0,1,1,1,3414,170,3
4978,0,66,1,0,1,0,0,1947,177,3
4979,1,61,0,0,1,1,0,1350,159,3


In [37]:
parameters = {'max_depth':list(range(50)), 'min_samples_leaf':list(range(2,50,1)),'criterion':["gini", "entropy"]}

et=time.time()
model= DecisionTreeClassifier()
clf = GridSearchCV(model, parameters)

clf.fit(X_train, y_train)
ft=time.time()

print("elapsed time {}".format(ft-et))
print("score: {}".format(clf.score(X_test,y_test)))

elapsed time 1837.2380537986755
score: 0.948494983277592


In [39]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(15,32)
        self.fc2=nn.Linear(32,64)
        self.fc6=nn.Linear(64,128)
        self.fc7=nn.Linear(128,256)
        self.fc8=nn.Linear(256,128)
        self.fc3=nn.Linear(128,32)
        self.fc4=nn.Linear(32,8)
        self.fc5=nn.Linear(8,1)


    def forward(self,x):
        
        x = F.relu(self.fc1(x))
        
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = F.relu(self.fc3(x))
        
        x = F.relu(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        return x
    
     
    
    
net = Net()
net.load_state_dict(torch.load(r'C:\Users\User\Desktop\מדמח\מודלימז\פרויקט חדש\batches117'))
#best_max_depth,best_min_samples_leaf,best_criterion=params[np.argmax(scores)]
#best_criterion='entropy' if best_criterion==1 else 'gini'

<All keys matched successfully>

In [60]:
scores=[]

et=time.time()
statistics=np.array(findStatistics(X_train,y_train))
for max_depth in list(range(50)):
    for min_samples_leaf in list(range(2,50,1)):
        for criterion in [1, 2]:
            hyperTensor=np.array([max_depth,min_samples_leaf,criterion])
            combtensor=torch.tensor(np.concatenate((statistics,hyperTensor))).float()
            
            score_predicted=net(combtensor).detach().numpy()
            scores.append(score_predicted)
params=[]
for max_depth in list(range(50)):
    for min_samples_leaf in list(range(2,50,1)):
        for criterion in [1, 2]:
            params.append((max_depth,min_samples_leaf,criterion)) 
            
scores=np.array(scores).flatten()
params=np.array(params)

tenLargetIdx = (-scores).argsort()[:10]
bests=params[tenLargetIdx]   


res=[]
for max_depth,min_samples_leaf,criterion in bests:
    criterion='entropy' if criterion==1 else 'gini'
    model= DecisionTreeClassifier(max_depth=max_depth,min_samples_leaf=min_samples_leaf,criterion=criterion)
    model.fit(X_train,y_train)
    res.append(model.score(X_test,y_test))
    

best=np.argmax(np.array(res))
best_max_depth,best_min_samples_leaf,best_criterion=bests[best]
best_criterion='entropy' if best_criterion==1 else 'gini'
model2= DecisionTreeClassifier(max_depth=best_max_depth,min_samples_leaf=best_min_samples_leaf,criterion=best_criterion)

model2.fit(X_train,y_train)
ft=time.time()
print("elapsed time {}".format(ft-et))

print('score: {}'.format(model2.score(X_test,y_test)))
            


elapsed time 10.09899353981018
score: 0.9438127090301003


In [67]:
best_max_depth

7